Исследование поведения пользователей в приложении и в рамках которого ответить на два вопроса:

1. Зависит ли вероятность оплаты от выбранного пользователем уровня сложности?
2. Существует ли разница во времени между событиями регистрации и оплаты для разных групп пользователей с разным уровнем сложности?

ЗАДАНИЕ
Необходимо проверить:
- есть ли зависимость между выбранным уровнем сложности и вероятностью оплаты;
- различается ли временной промежуток между регистрацией и оплатой у групп пользователей с разным уровнем сложности.

Важно! Выполнять задание необходимо на основе данных пользователей, которые зарегистрировались в 2018 году (с 1 января по 31 декабря 2018 года включительно).

In [3]:
import pandas as pd

# Загрузка всех файлов проекта и создание их копий для дальнейших преобразований

event = pd.read_csv('7_4_Events.csv')
events_df = event.copy()


Проверка файла events_df

In [5]:
# Обзор таблицы events_df
events_df.head(10)

,id,event_type,selected_level,start_time,tutorial_id,user_id
0,28903,registration,NaN,2016-05-11T23:40:55,NaN,12583
1,28904,registration,NaN,2016-05-11T23:49:58,NaN,12584
2,28905,registration,NaN,2016-05-12T00:53:07,NaN,12585
3,28906,tutorial_start,NaN,2016-05-12T01:32:20,17562.0,12585
4,28907,tutorial_finish,NaN,2016-05-12T01:34:53,17562.0,12585
5,28908,tutorial_start,NaN,2016-05-12T02:11:29,17563.0,12584
6,28909,tutorial_finish,NaN,2016-05-12T02:14:14,17563.0,12584
7,28910,level_choice,hard,2016-05-12T05:30:25,NaN,12584
8,28911,pack_choice,NaN,2016-05-12T05:38:24,NaN,12584
9,28912,tutorial_start,NaN,2016-05-12T07:19:01,17564.0,12583


In [6]:
# Информация о столбцах events_df
events_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252334 entries, 0 to 252333
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              252334 non-null  int64  
 1   event_type      252334 non-null  object 
 2   selected_level  31086 non-null   object 
 3   start_time      252334 non-null  object 
 4   tutorial_id     125103 non-null  float64
 5   user_id         252334 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 11.6+ MB


In [7]:
# Перевод столбца start_time в формат дат

events_df['start_time'] = pd.to_datetime(events_df['start_time'], errors='coerce')

# Проверка перобразования
events_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252334 entries, 0 to 252333
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   id              252334 non-null  int64         
 1   event_type      252334 non-null  object        
 2   selected_level  31086 non-null   object        
 3   start_time      252201 non-null  datetime64[ns]
 4   tutorial_id     125103 non-null  float64       
 5   user_id         252334 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 11.6+ MB


Выполнять задание необходимо на основе данных пользователей, которые зарегистрировались в 2018 году (с 1 января по 31 декабря 2018 года включительно).

In [8]:
# Маска по пользователям зарегестрированным в 2018 г.

mask2018 = (events_df['start_time'] < '2019-01-01') & (events_df['start_time'] >= '2018-01-01') & (events_df['event_type'] == 'registration')

# Фильтрация таблийы с данными пользователей зарегестрированными в 2018 г.
user_list = list(events_df[mask2018]['user_id'])
events_df = events_df[events_df['user_id'].isin(user_list)]

In [9]:
# Уникальные события в стобце event_type
events_df['event_type'].unique()

array(['registration', 'tutorial_start', 'tutorial_finish',
       'level_choice', 'pack_choice'], dtype=object)

Проверка файла purchase_df

In [10]:
# Загрузка файла проекта и создание его копии для дальнейших преобразований

purchase = pd.read_csv('purchase.csv')
purchase_df = purchase.copy()

In [11]:
purchase_df.head(10)

,id,user_id,event_datetime,amount
0,15674,12584,2016-05-12T10:34:16,100
1,15675,12985,2016-05-13T08:25:56,50
2,15676,12828,2016-05-13T16:33:46,50
3,15677,12598,2016-05-14T01:09:37,150
4,15678,13037,2016-05-14T01:24:46,100
5,15679,12989,2016-05-14T02:19:20,200
6,15680,12684,2016-05-14T03:10:14,100
7,15681,12656,2016-05-14T03:42:48,100
8,15682,12624,2016-05-14T06:00:45,50
9,15683,13136,2016-05-14T07:31:03,50


In [12]:
# Информация по столбцам таблицы purchase_df
purchase_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5956 entries, 0 to 5955
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              5956 non-null   int64 
 1   user_id         5956 non-null   int64 
 2   event_datetime  5956 non-null   object
 3   amount          5956 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 186.2+ KB


In [13]:
# Столбик event_datetime в формат дат

purchase_df['event_datetime'] = pd.to_datetime(purchase_df['event_datetime'])

In [14]:
# Фильтрация по purchase_df по списку пользователей зарегестрированх в 2018 г.
purchase_df = purchase_df[purchase_df['user_id'].isin(user_list)]

In [15]:
# Проверка преобразования
purchase_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1600 entries, 1171 to 2778
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              1600 non-null   int64         
 1   user_id         1600 non-null   int64         
 2   event_datetime  1600 non-null   datetime64[ns]
 3   amount          1600 non-null   int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 62.5 KB


ОБЪЕДИНЕНИЕ ДАТАФРЕМОВ

In [16]:
total_events_df = pd.concat([events_df,purchase_df],sort=False)

In [17]:
total_events_df.head(10)

,id,event_type,selected_level,start_time,tutorial_id,user_id,event_datetime,amount
51405,80308,registration,NaN,2018-01-01 03:48:40,NaN,27832,NaT,NaN
51406,80309,registration,NaN,2018-01-01 04:07:25,NaN,27833,NaT,NaN
51407,80310,registration,NaN,2018-01-01 08:35:10,NaN,27834,NaT,NaN
51408,80311,registration,NaN,2018-01-01 11:54:47,NaN,27835,NaT,NaN
51409,80312,registration,NaN,2018-01-01 13:28:07,NaN,27836,NaT,NaN
51410,80313,registration,NaN,2018-01-01 14:08:40,NaN,27837,NaT,NaN
51411,80314,registration,NaN,2018-01-01 14:42:58,NaN,27838,NaT,NaN
51412,80315,tutorial_start,NaN,2018-01-01 14:54:40,31505.0,27836,NaT,NaN
51413,80316,tutorial_start,NaN,2018-01-01 15:00:51,31506.0,27835,NaT,NaN
51414,80317,tutorial_finish,NaN,2018-01-01 15:06:15,31506.0,27835,NaT,NaN


In [18]:
total_events_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68559 entries, 51405 to 2778
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              68559 non-null  int64         
 1   event_type      66959 non-null  object        
 2   selected_level  8342 non-null   object        
 3   start_time      66959 non-null  datetime64[ns]
 4   tutorial_id     32954 non-null  float64       
 5   user_id         68559 non-null  int64         
 6   event_datetime  1600 non-null   datetime64[ns]
 7   amount          1600 non-null   float64       
dtypes: datetime64[ns](2), float64(2), int64(2), object(2)
memory usage: 4.7+ MB


In [19]:
# Сброс индексов (что бы не было дуюликатов)
total_events_df = total_events_df.reset_index(drop=True).sort_values('start_time')

ОПРЕДЕЛЕНИЕ ГРУПП ПОЛЬЗОВАТЕЛЕЙ ПО ВЫБРАНОМУ УРОВНЮ СЛОЖНОСТИ

In [20]:
# Определим типы уровней сложности и типы событий
display(total_events_df['selected_level'].unique())
display(total_events_df['event_type'].value_counts())
display(total_events_df['selected_level'].value_counts())

array([nan, 'medium', 'hard', 'easy'], dtype=object)

registration       19926
tutorial_start     18050
tutorial_finish    14904
level_choice        8342
pack_choice         5737
Name: event_type, dtype: int64

medium    4645
easy      2448
hard      1249
Name: selected_level, dtype: int64

In [21]:
# Создадим датафрем с полльзователями сделавшими выбор уровня сложности
level_choise_users = total_events_df[total_events_df['event_type'] == 'level_choice']

# Посмотрим на данные нового датафрейма level_choise_users
display(level_choise_users.head())
display(level_choise_users.info())
display(level_choise_users.describe())

,id,event_type,selected_level,start_time,tutorial_id,user_id,event_datetime,amount
19,80327,level_choice,medium,2018-01-01 20:37:22,NaN,27835,NaT,NaN
23,80331,level_choice,hard,2018-01-01 22:37:50,NaN,27839,NaT,NaN
36,80344,level_choice,medium,2018-01-02 05:18:42,NaN,27840,NaT,NaN
38,80346,level_choice,hard,2018-01-02 06:19:18,NaN,27845,NaT,NaN
45,80353,level_choice,easy,2018-01-02 08:46:03,NaN,27842,NaT,NaN


<class 'pandas.core.frame.DataFrame'>
Int64Index: 8342 entries, 19 to 66957
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              8342 non-null   int64         
 1   event_type      8342 non-null   object        
 2   selected_level  8342 non-null   object        
 3   start_time      8342 non-null   datetime64[ns]
 4   tutorial_id     0 non-null      float64       
 5   user_id         8342 non-null   int64         
 6   event_datetime  0 non-null      datetime64[ns]
 7   amount          0 non-null      float64       
dtypes: datetime64[ns](2), float64(2), int64(2), object(2)
memory usage: 586.5+ KB


None

,id,tutorial_id,user_id,amount
count,8342.000000,0.0,8342.000000,0.0
mean,113973.378566,NaN,37828.492208,NaN
std,19397.260408,NaN,5771.799004,NaN
min,80327.000000,NaN,27835.000000,NaN
25%,97228.250000,NaN,32882.250000,NaN
50%,113945.500000,NaN,37747.500000,NaN
75%,131005.000000,NaN,42873.750000,NaN
max,147266.000000,NaN,47756.000000,NaN


In [22]:
# Проверка на то, что на 1 пользователя приходится 1 уровень сложности
display(level_choise_users.shape[0] == level_choise_users['user_id'].nunique())


True

1. Зависит ли вероятность оплаты от выбранного пользователем уровня сложности?

In [23]:
# Готовим списки пользователей по уровням easy, medium и hard

# easy
users_easy_level = level_choise_users[
    level_choise_users['selected_level'] == 'easy'
]['user_id']
display('Число пользоветлям по уровням сложности:')
display('Easy: {}'.format(len(users_easy_level)))

# medium
users_medium_level = level_choise_users[
    level_choise_users['selected_level'] == 'medium'
]['user_id']
display('Medium: {}'.format(len(users_medium_level)))

# hard
users_hard_level = level_choise_users[
    level_choise_users['selected_level'] == 'hard'
]['user_id']
display('Hard: {}'.format(len(users_hard_level)))

'Число пользоветлям по уровням сложности:'

'Easy: 2448'

'Medium: 4645'

'Hard: 1249'

In [24]:
# Сколько пользователей уровня easy оплатили обучение?
purchase_df_easy = purchase_df[purchase_df["user_id"].isin(users_easy_level)]
display('Чисило пользователей уровня easy оплативших обучение: {}'.format(purchase_df_easy.shape[0]))


# Какой процент пользователей easy оплачивает обучение?
percent_easy_paid = purchase_df_easy.shape[0]/len(users_easy_level)
display('% пользователей уровня easy оплативших обучение: {:.2%}'.format(percent_easy_paid))

# Сколько % пользователей уровня easy оплатили обучение от общего числа оплативших всех уровней?
percent_easy_paid_from_all = purchase_df_easy.shape[0]/purchase['user_id'].nunique()
display('% пользователей уровня easy оплатившие обучение от общего числа всех оплативших: {:.2%}'.format(percent_easy_paid_from_all))



'Чисило пользователей уровня easy оплативших обучение: 189'

'% пользователей уровня easy оплативших обучение: 7.72%'

'% пользователей уровня easy оплатившие обучение от общего числа всех оплативших: 3.17%'

In [25]:
# Сколько пользователей уровня medium оплатили обучение?
purchase_df_medium = purchase_df[purchase_df["user_id"].isin(users_medium_level)]
display('Чисило пользователей уровня medium оплативших обучение: {}'.format(purchase_df_medium.shape[0]))

# Какой процент пользователей medium оплачивает обучение?
percent_medium_paid = purchase_df_medium.shape[0]/len(users_medium_level)
display('% пользователей уровня medium оплативших обучение: {:.2%}'.format(percent_medium_paid))

# Сколько % пользователей уровня medium оплатили обучение от общего числа оплативших всех уровней?
percent_medium_paid_from_all = purchase_df_medium.shape[0]/purchase['user_id'].nunique()
display('% пользователей уровня medium оплатившие обучение от общего числа всех оплативших: {:.2%}'.format(percent_medium_paid_from_all))

'Чисило пользователей уровня medium оплативших обучение: 969'

'% пользователей уровня medium оплативших обучение: 20.86%'

'% пользователей уровня medium оплатившие обучение от общего числа всех оплативших: 16.27%'

In [26]:
# Сколько пользователей уровня hard оплатили обучение?
purchase_df_hard = purchase_df[purchase_df["user_id"].isin(users_hard_level)]
display('Чисило пользователей уровня hard оплативших обучение: {}'.format(purchase_df_hard.shape[0]))

# Какой процент пользователей hard оплачивает обучение?
percent_hard_paid = purchase_df_hard.shape[0]/len(users_hard_level)
display('% пользователей уровня easy оплативших обучение: {:.2%}'.format(percent_hard_paid))

# Сколько % пользователей уровня hard оплатили обучение от общего числа оплативших всех уровней?
percent_hard_paid = purchase_df_hard.shape[0]/purchase['user_id'].nunique()
display('% пользователей уровня hard оплатившие обучение от общего числа всех оплативших: {:.2%}'.format(percent_hard_paid))

'Чисило пользователей уровня hard оплативших обучение: 442'

'% пользователей уровня easy оплативших обучение: 35.39%'

'% пользователей уровня hard оплатившие обучение от общего числа всех оплативших: 7.42%'

ВЫВОДЫ ПО ВОПРОСУ №1

1. На уровне easy наблюдается нименщее число пользоватей и наименьщая вероятность опаты, которая составляет 7.72%.

2. Наибольшее число пользователей наблюдается на уровне medium, соответсвено основной доход приносит эта группа, пользователи выбравшие уровнеь medium производят оплату с вероятностю 20.86%.

3. Пользователи уровня hard производят оплату с наибольшей вероятностью 35.39%.

4. Наблюдается прямая зависимость между выбранным уровнем сложности и вероятностью оплаты, чем выше уровень сложности, тем выше вероятность оплаты. Рекомендуется привлекать пользлвтелей на уровень hard(доп бонусами или возможностями) и добавить уровень extra hard, возможно это повысить количествопользователей на уровне hard.

2. Существует ли разница во времени между событиями регистрации и оплаты для разных групп пользователей с разным уровнем сложности?

In [42]:
# Создадим датафрем с датой регистрации и уберем лишние стобцы.
registration_df = total_events_df[total_events_df['event_type'] == 'registration']

registration_df = registration_df[['user_id', 'selected_level', 'start_time']].rename(
    columns={"start_time": "registration_time"})

registration_df.head()

,user_id,selected_level,registration_time
0,27832,NaN,2018-01-01 03:48:40
1,27833,NaN,2018-01-01 04:07:25
2,27834,NaN,2018-01-01 08:35:10
3,27835,NaN,2018-01-01 11:54:47
4,27836,NaN,2018-01-01 13:28:07


In [55]:
purchase_df.head()

,id,user_id,event_datetime,amount
1171,16845,27845,2018-01-03 18:53:43,100
1172,16846,27865,2018-01-04 14:46:10,250
1174,16848,27911,2018-01-07 08:19:12,50
1175,16849,27910,2018-01-07 12:11:34,100
1176,16850,27940,2018-01-07 13:16:41,200


In [57]:
# Создадим новый purchace датафрейм для объединения без лишних стобцов.
purchase_df2 = purchase_df[["user_id", "event_datetime"]].rename(
    columns={"event_datetime": "purchase_time"}
)
purchase_df2.head()

,user_id,purchase_time
1171,27845,2018-01-03 18:53:43
1172,27865,2018-01-04 14:46:10
1174,27911,2018-01-07 08:19:12
1175,27910,2018-01-07 12:11:34
1176,27940,2018-01-07 13:16:41


In [73]:
# Объединяю датафрем с датой регистрации и датой покупки по user_id
merged_df = registration_df.merge(
    purchase_df2, on='user_id', how='inner'
)
display(merged_df.head())

,user_id,selected_level,registration_time,purchase_time
0,27845,NaN,2018-01-02 01:35:56,2018-01-03 18:53:43
1,27865,NaN,2018-01-03 11:14:57,2018-01-04 14:46:10
2,27884,NaN,2018-01-04 11:50:43,2018-01-08 19:37:34
3,27910,NaN,2018-01-05 10:45:33,2018-01-07 12:11:34
4,27911,NaN,2018-01-05 10:48:24,2018-01-07 08:19:12


In [76]:
# Построчно посчитаем временой интервал от регистрации до покупки и занесем с столбик timedelta
merged_df["timedelta"] = (
    merged_df["purchase_time"] - merged_df["registration_time"]
)
merged_df.head()

,user_id,selected_level,registration_time,purchase_time,timedelta
0,27845,NaN,2018-01-02 01:35:56,2018-01-03 18:53:43,1 days 17:17:47
1,27865,NaN,2018-01-03 11:14:57,2018-01-04 14:46:10,1 days 03:31:13
2,27884,NaN,2018-01-04 11:50:43,2018-01-08 19:37:34,4 days 07:46:51
3,27910,NaN,2018-01-05 10:45:33,2018-01-07 12:11:34,2 days 01:26:01
4,27911,NaN,2018-01-05 10:48:24,2018-01-07 08:19:12,1 days 21:30:48


Посчитаем время от регистрации до оплаты по группам разбитым на разные уровни сложности: easy, medium, hard

In [83]:
# easy
merged_df[merged_df['user_id'].isin(purchase_df_easy['user_id'])].describe()

,user_id,timedelta
count,189.000000,189
mean,38083.476190,3 days 22:10:23.211640211
std,6160.867379,2 days 07:14:41.062010764
min,27884.000000,0 days 04:36:58
25%,32645.000000,2 days 01:12:12
50%,37658.000000,3 days 11:00:23
75%,43870.000000,5 days 10:24:59
max,47687.000000,11 days 00:35:04


In [84]:
# medium
merged_df[merged_df['user_id'].isin(purchase_df_medium['user_id'])].describe()

,user_id,timedelta
count,969.000000,969
mean,37740.134159,4 days 06:12:06.576883384
std,5731.786288,2 days 06:25:57.480868026
min,27973.000000,0 days 08:39:24
25%,32896.000000,2 days 08:46:51
50%,37533.000000,4 days 03:35:26
75%,42980.000000,5 days 23:51:27
max,47742.000000,10 days 20:34:02


In [85]:
# hard
merged_df[merged_df['user_id'].isin(purchase_df_hard['user_id'])].describe()

,user_id,timedelta
count,442.000000,442
mean,37639.047511,3 days 14:55:19.257918552
std,5879.821921,1 days 22:22:52.441896774
min,27845.000000,0 days 09:41:39
25%,32564.500000,1 days 23:36:25.500000
50%,37739.000000,3 days 10:10:04.500000
75%,42798.500000,5 days 03:30:07.750000
max,47577.000000,8 days 14:21:29


1. Пользователи группы easy в среднем после регистрации производят оплату через 3 дня 22 ч и 10 мин.
2. Пользователи группы medium в среднем после регистрации производят оплату через 4 дня 6 ч и 12 мин.
3. Пользователи группы hard в среднем после регистрации производят оплату через 3 дня 14 ч и 55 мин.
Разница между группами небольшая, наиболее быстро производит оплату группа hard.

ВЫВОДЫ ПО ВОПРОСАМ:

1. Вероятность опалты напрямуб зависит от выбранного уровня сложности: easy - 7,72%, medium - 20,86%, hard - 35,39%. При этом группа medium самой многочисленной.
2. Существует разница во времени в скорости оплаты с момонета регистрации для разных групп пользователей, но при этом разница не сущетсвенная и не наблюдается зависимости скорости опалты от уровня сложности.

РЕКОМЕНДАЦИИ:
1. Повысить привлекателньость уровня hard (дополнительные бонусы или другое). 
2. Добавить уровень сложности Extra hard как стимул регистрироваться на уровне hard, большему числу пользователей.
3. Проанализировать поэтапно скорость прохождения пути от регистрации до оплаты на разных уровнях, для выявления самых долгих этапов.